# Copyright 2022 Cognite AS
## Authentication

- Instatiating a `CogniteClient` can be done using several different options:
  - Interactive login (send you to a browser to log in)
  - Using device code (prints out a code you need to use to log in elsewhere)
  - Using client credentials (pre-shared secret)
- Checking the login status

You can authenticate the Python SDK with Azure AD by using a token retrieved when a user authenticates or with a static client secret for long-running jobs like extractors or calculations.

### Prerequisites

* You need to specify the values for the following configuration parameters:
 * `Tenant ID` - the ID of the Azure AD tenant where the user is registered.
 * `Client ID` - the ID of the application in Azure AD.
 * `Cluster` - the cluster where your CDF project is installed. For example, `api` or `westeurope-1`.
 * `CDF project` - the name of the CDF project, typically the name of your company.

If you don't know which values to use for these variables, contact your CDF administrator or Cognite Support.

You can directly set the values for these parameters here. Another option is to read them from environment variables or from file.

In [ ]:
TENANT_ID = "48d5043c-cf70-4c49-881c-c638f5796997"
CLIENT_ID = "fab52bb5-9de2-4f9e-aefa-712da4b5fe00"
CDF_CLUSTER = "westeurope-1"
COGNITE_PROJECT = "ds-basics"

Also set the following derived variables, which will be used for obtaining token

In [ ]:
BASE_URL = f"https://{CDF_CLUSTER}.cognitedata.com"
SCOPES = [f"https://{CDF_CLUSTER}.cognitedata.com/.default"]
TOKEN_URL = f"https://login.microsoftonline.com/{TENANT_ID}/oauth2/v2.0/token"
AUTHORITY_URL = f"https://login.microsoftonline.com/{TENANT_ID}"

### Authenticate to CDF

#### Interactive Login
You can get the token by letting the user sign in interactively via a browser and use the authenticate with interactive login and token refresh flow to access CDF when you're running short-term scripts or using Jupyter.

In [ ]:
from cognite.client import CogniteClient, ClientConfig
from cognite.client.credentials import OAuthInteractive

In [ ]:
interactive_creds = OAuthInteractive(
    authority_url=AUTHORITY_URL,
    client_id=CLIENT_ID,
    scopes=SCOPES,
)
cnf = ClientConfig(
    client_name="Python SDK course taker",
    project=COGNITE_PROJECT,
    credentials=interactive_creds,
    base_url=BASE_URL,
)

In [ ]:
client = CogniteClient(cnf)

We can now check out which CDF projects we have access to:

In [ ]:
token_inspect = client.iam.token.inspect()

[proj.url_name for proj in token_inspect.projects]

### Authenticate with client secret

In [ ]:
from cognite.client.credentials import OAuthClientCredentials

The secret (called a client secret) you got from the course admins can be used to authenticate without being sent off to the browser to login on your own.

Store it in your preferred secret manager, for example LastPass (or similar).

In this notebook, we will provide it by using the python function `getpass`. 

**Note**: This is _not_ how one would do it for code running in production. Then it is typically read from an environment variable or from a file.

In [ ]:
from getpass import getpass
CLIENT_SECRET = getpass("Enter the Client Secret: ")

In [ ]:
client_creds = OAuthClientCredentials(
    token_url=TOKEN_URL, 
    client_id=CLIENT_ID, 
    scopes=SCOPES, 
    client_secret=CLIENT_SECRET
)
cnf = ClientConfig(
    client_name="Python SDK course taker",
    project=COGNITE_PROJECT,
    credentials=client_creds,
)

In [ ]:
client = CogniteClient(cnf)

As previously, we check if we are authenticated by calling the `/token/inspect` endpoint:

In [ ]:
token_inspect = client.iam.token.inspect()

[proj.url_name for proj in token_inspect.projects]

### Saving the Authentication Code in Script for Later Usage

Let's define the function for each authentication method.

In [ ]:
def create_cognite_client():
    """Function to create the CogniteClient, using interactive login flow"""
    return CogniteClient(
        ClientConfig(
            client_name="Python SDK course taker",
            project=COGNITE_PROJECT,
            base_url=BASE_URL,
            credentials=OAuthInteractive(
                authority_url=AUTHORITY_URL,
                client_id=CLIENT_ID,
                scopes=SCOPES,
            ),
        )
    )

#### With this function created, going forward in this course, we can easily get the client :

In [ ]:
client = create_cognite_client()